In [2]:
import numpy as np
import itertools

In [77]:
def apply_simulation_parameters_mutation(individual, mutation_rate, means, std_devs, min_vals, max_vals, distribution):
 
    mut_mask = np.random.rand(2) < mutation_rate
    
    for i in range(2):
        
        if distribution == "uniform":
            individual[-1, -1, i+3] += (np.random.uniform(low=min_vals[i], high=max_vals[i]) - individual[-1, -1, i+3]) * mut_mask[i]
        elif distribution == "normal":
            individual[-1, -1, i+3] += np.random.normal(loc=means[i], scale=std_devs[i]) * mut_mask[i]
        
        individual[-1, -1, i+3] = max(min_vals[i], min(max_vals[i], individual[-1, -1, i+3]))

    return individual


In [78]:
for i in range(20):
    ind = np.zeros((3, 10, 10))
    ind[-1, -1, 3:5] = [20, .2]
    ind = apply_simulation_parameters_mutation(ind, 0.9, [10, .1], [5, .01], [5, .00001], [100, .1], "uniform")
    print(ind[-1, -1, :])

[ 0.          0.          0.         37.52759172  0.05441912  0.
  0.          0.          0.          0.        ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 4.30413463e+01
 3.28390036e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 7.28250926e+00
 7.24989972e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[ 0.          0.          0.         89.81335426  0.1         0.
  0.          0.          0.          0.        ]
[ 0.          0.          0.         17.34162218  0.04184204  0.
  0.          0.          0.          0.        ]
[ 0.          0.          0.         20.          0.02352531  0.
  0.          0.          0.          0.        ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.85644611e+01
 5.46912956e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[ 0.          0.          0.         20.          0.06251275  0.
  0.         

In [111]:
def apply_compartment_mutation(individual, mutation_rate, mean, std_dev, min_val, max_val, distribution):

    num_species = int(individual[-1, -1, 0])
    z, y, x = individual.shape
   
    for i in range(1, num_species * 2, 2):
        mut_mask = np.random.rand(y, x) < mutation_rate

        if distribution == "normal":
            noise = np.random.normal(loc=mean, scale=std_dev, size=(y, x))
        elif distribution == "uniform":
            noise = np.random.uniform(low=min_val, high=max_val, size=(y, x))

        individual[i, :, :] += np.where(mut_mask, noise, 0)
        individual[i, :, :] = np.clip(individual[i, :, :], min_val, max_val)

    return individual


In [112]:
ind = np.zeros((4, 5, 5))
ind[-1, -1, 0] = 2

In [113]:
ind = apply_compartment_mutation(ind, .3, 10, 3, 0, 100, "normal")

In [114]:
ind

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[14.0717607 ,  0.        , 10.37036457,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
         16.64442524],
        [10.76495102,  0.        , 10.53249029,  0.        ,
          0.        ],
        [ 0.        , 10.38751566,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  9.13504115,  0.        ,
          9.68256362]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0

In [137]:
def apply_parameter_mutation(individual, mutation_rate, species_means, species_std_devs, species_min_vals, species_max_vals, 
                             complex_means, complex_std_devs, complex_min_vals, complex_max_vals, distribution):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = num_species * 2
    pair_stop = pair_start + (num_pairs * 2)

    count = 0
    for i in range(0, num_species * 2, 2):
        mut_mask = np.random.rand(3) < mutation_rate
        if distribution == "normal":
            for j in range(3):
                individual[-1, i, j] += np.random.normal(loc=species_means[count], scale=species_std_devs[count]) * mut_mask[j]
        elif distribution == "uniform":
            for j in range(3):
                individual[-1, i, j] += (np.random.uniform(low=species_min_vals[count], high=species_max_vals[count]) - individual[-1, i, j]) * mut_mask[j]
        count += 1
        
    for i in range(pair_start+1, pair_stop, 2):
        mut_mask = np.random.rand(4) < mutation_rate
        if distribution == "normal":
            for j in range(4):
                individual[i, 1, j] += np.random.normal(loc=complex_means[count], scale=complex_std_devs[count]) * mut_mask[j]
        elif distribution == "uniform":
            for j in range(4):
                individual[i, 1, j] += (np.random.uniform(low=complex_min_vals[count], high=complex_max_vals[count]) - individual[i, 1, j]) * mut_mask[j]
        count += 1

    return individual

In [138]:
ind = np.zeros((7, 10, 10))
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 81]

In [139]:
for i in range(20):
    ind = np.zeros((9, 5, 5))
    ind[-1, -1, 0] = 2
    ind[-1, -1, 1] = 2
    ind[-1, 0, :3] = [.2, .5, .7]
    ind[-1, 2, :3] = [.2, .5, .7]
    ind[-4, 1, :4] = [20, 10, 30, 80]
    ind[-2, 1, :4] = [20, 10, 30, 80]
    ind = apply_parameter_mutation(ind, .9, [1, 1, 1], [.2, .2, .3], [0, 0, 0], [5, 5, 5], 
                                   [3,4,5,5], [1,1,1,1], [0, 0, 0, 0], [10,10,10,10],"uniform")
    print(ind[-2:, :,:])

[[[ 0.          0.          0.          0.          0.        ]
  [ 3.62373921 10.          6.43852797  3.88023318  0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 3.23656519  0.27588055  1.3067239   0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.59899239  1.21898785  0.98568723  0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 2.          2.          0.          0.          0.        ]]]
[[[0.         0.         0.         0.         0.        ]
  [8.12369316 5.90068889 8.00768263 2.07185333 0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[3.52875232 4.71259768 1.11572155 0.         0.        ]


In [52]:
import itertools

def apply_species_insertion_mutation(individual, mutation_rate):


    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    z, y, x = individual.shape

    if np.random.rand() < mutation_rate:
        pairs = pair_finding(
            num_species=num_species
        )
        init_matrix = species_initialization(
            compartment_size=(y, x),
            pairs=pairs
        )
        individual = species_combine(
            individual=individual,
            init_matrix=init_matrix,
            num_species=num_species,
            num_pairs=num_pairs
        )

    return individual



def apply_species_deletion_mutation(individual, mutation_rate):
 
    num_species = int(individual[-1, -1, 0])

    if np.random.rand() < mutation_rate and num_species > 1:
        deleted_species = int(np.random.choice(np.arange(2, num_species+1)))

        individual = species_deletion(
            individual=individual,
            deleted_species=deleted_species
        )

    return individual




def pair_finding(num_species):


    last = num_species + 1
    species = [i for i in range(1, num_species + 2, 1)]
    pairs = list(itertools.combinations(species, 2))

    related_pairs = [pair for pair in pairs if last in pair]
    pair_indices = [((pair[0] - 1) * 2, (pair[1]-1)*2) for pair in related_pairs]

    return pair_indices


def species_initialization(compartment_size, pairs):


    num_species = len(pairs) + 1
    num_matrices = num_species * 2
    init_matrix = np.zeros((num_matrices, compartment_size[0], compartment_size[1]))

    for i in range(len(pairs)):
        m = np.zeros((2, compartment_size[0], compartment_size[1]))
        m[-1, 0, 0] = int(pairs[i][0])
        m[-1, 0, 1] = int(pairs[i][1])
        m[-1, 1, :4] = np.random.rand(4)
        init_matrix[i*2+2:i*2+4, :, :] = m

    return init_matrix



def species_combine(individual, init_matrix, num_species, num_pairs):
 

    z, y, x = individual.shape
    z1 = z + init_matrix.shape[0]

    updated_individual = np.zeros((z1, y, x))
    updated_individual[:num_species * 2, :, :] = individual[:num_species * 2, :, :]
    updated_individual[num_species * 2:num_species * 2 + init_matrix.shape[0], :, :] = init_matrix
    updated_individual[num_species * 2 + init_matrix.shape[0]:, :, :] = individual[num_species * 2:, :, :]
    updated_individual[-1, -1, 0] = int(num_species + 1)
    updated_individual[-1, -1, 1] = int(num_pairs + ((init_matrix.shape[0] - 2) / 2))
    updated_individual[-1, num_species * 2, :3] = np.random.rand(3)

    return updated_individual



def species_deletion(individual, deleted_species):
   
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int((num_species * 2) + 1)
    pair_stop = int(pair_start + (num_pairs * 2))

    delete_indices = [(deleted_species-1)*2, ((deleted_species-1)*2)+1]

    for i in range(pair_start, pair_stop, 2):
        if int((individual[i, 0, 0] / 2) + 1) == deleted_species or int((individual[i, 0, 1] / 2) + 1) == deleted_species:
            delete_indices.extend([i - 1, i])
    print(delete_indices)

    updated_individual = np.delete(individual, delete_indices, axis=0)

    updated_individual[-1, -1, 0] = num_species - 1
    updated_individual[-1, -1, 1] = num_pairs - len(delete_indices) // 2 + 1

    return updated_individual


In [53]:
ind = np.zeros((7, 10, 10))
ind[-1, -1, 0] = 2
ind[-1, -1, 1] = 1
ind[-2, 0, :2] = [0, 2]
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 80]
ind[0, 5, 5] = 1
ind[1, 5, 5] = 1

print(ind.shape)  # Original shape: (7, 10, 10)

ind = apply_species_insertion_mutation(ind, 0.9)
print(ind.shape)  # After insertion mutation

ind = apply_species_deletion_mutation(ind, 0.95)
print(ind.shape) 

(7, 10, 10)
(7, 10, 10)
[2, 3, 4, 5]
(3, 10, 10)


In [54]:
ind

array([[[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]],

       [[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ,

In [3]:
def apply_simulation_variable_crossover(elite_individual, individual, alpha):

    individual[-1, -1, 3:5] = (alpha * individual[-1, -1, 3:5]) + ((1 - alpha) * elite_individual[-1, -1, 3:5])

    return individual

In [4]:
ind = np.zeros((7, 10, 10))
ind1 = np.zeros((7, 10, 10))
ind[-1, -1, 3:5] = [20, .02]
ind1[-1, -1, 3:5] = [50, .001]
inds = apply_simulation_variable_crossover(ind1, ind, 0.4)
inds[-1, :, :]

array([[0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
        0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 3.

In [5]:
print(8.6e-03)

0.0086


In [6]:
print(3.8e+01)

38.0


In [7]:
ind1[-1, :, :]

array([[0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 5.e+01, 1.e-03, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00]])

In [15]:
def apply_compartment_crossover(elite_individual, individual, alpha):
    num_species = int(individual[-1, -1, 0])

    for i in range(1, num_species*2+1, 2):
        individual[i, :, :] = (alpha * individual[i, :, :]) + ((1 - alpha) * elite_individual[i, :, :])

    return individual

In [16]:
ind = np.zeros((7, 10, 10))
ind1 = np.zeros((7, 10, 10))
ind[-1, -1, 0] = 2
ind[1, :, :] = np.ones((10, 10))
ind[3, 4:7, :] = np.full((3, 10), fill_value=3.2)
ind1[1, :, :] = np.full((10, 10), fill_value=0.6)
ind1[3, :, :] = np.full((10, 10), fill_value=0.1)

In [17]:
print(ind)
print(ind1)
ind = apply_compartment_crossover(ind1, ind, 0.7)

[[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]]

 [[1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
  [1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]]

 [[0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0

In [18]:
ind

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88],
        [0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88],
        [0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88],
        [0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88],
        [0.88, 0.8

In [32]:
def apply_parameter_crossover(elite_individual, individual, alpha):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int(num_species * 2)
    pair_stop = int(pair_start + (num_pairs * 2))

    for i in range(0, num_species*2, 2):
        individual[-1, i, :3] = (alpha * individual[-1, i, :3]) + ((1 - alpha) * elite_individual[-1, i, :3])

    for i in range(pair_start+1, pair_stop+1, 2):
        individual[i, 1, :4] = (alpha * individual[i, 1, :4]) + ((1 - alpha) * elite_individual[i, 1, :4])

    return individual

In [35]:
ind = np.zeros((7, 10, 10))
ind1 = np.zeros((7, 10, 10))
ind[-1, 0, :3] = [1, 2, 3]
ind1[-1, 0, :3] = [100, 200, 300]
ind[-1, 2, :3] = [12, 23, 34]
ind1[-1, 2, :3] = [10, 20, 30]
ind[-1, -1, 0] = 2
ind[-1, -1, 1] = 1
ind[-2, 1, :4] = [0.1, 0.3, 0.5, 89]
ind1[-2, 1, :4] = [1000, 1000, 1000, 10000]

In [36]:
print(ind)
print(ind1)

[[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.

In [37]:
inds = apply_parameter_crossover(ind1, ind, 0.2)

In [38]:
inds

array([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000

In [41]:
print(8.0200e+01, 1.6040e+02, 2.4060e+02)
print(1.0400e+01, 2.0600e+01, 3.0800e+01)
print(8.0002e+02, 8.0006e+02, 8.0010e+02, 8.0178e+03)

80.2 160.4 240.6
10.4 20.6 30.8
800.02 800.06 800.1 8017.8


In [64]:
def population_initialization(population_size, individual_shape, species_parameters, complex_parameters, num_species, num_pairs, max_sim_epochs, sim_stop_time, time_step, individual_fix_size):

    pair_start = int(num_species * 2)
    pair_stop = int(pair_start + (num_pairs * 2))
    population = []

    if individual_fix_size:
        population = [np.zeros(individual_shape) for _ in range(population_size)]

        for i in range(0, len(species_parameters) * 2, 2):
            print(i//2)
            for ind in population:
                ind[-1, i, :3] = species_parameters[int(i // 2)]

        for i in range(pair_start + 1, pair_stop + 1, 2):
            print(int((i-(pair_start+1))//2))
            for ind in population:
                ind[i, 0, :2] = complex_parameters[int((i-(pair_start+1))//2)][0]
                ind[i, 1, :4] = complex_parameters[int((i-(pair_start+1))//2)][1]

        for ind in population:
            ind[-1, -1, :5] = [num_species, num_pairs, max_sim_epochs, sim_stop_time, time_step]

    else:
        population = [np.zeros((3, individual_shape[1], individual_shape[2])) for _ in range(population_size)]
        for ind in population:
            ind[1, :, :] = np.random.rand(individual_shape[1], individual_shape[2])
            ind[-1, 0, :3] = np.random.rand(3)
            ind[-1, -1, :5] = [1, 0, max_sim_epochs, sim_stop_time, time_step]

    return population




population_size = 10
individual_shape = (13, 10, 10)  
species_parameters = [
    [0.1, 0.05, 0.02],  
    [0.2, .3, 0.12],
    [1, .6, 0.002]
]
complex_parameters = [
    [[1, 2], [1, 2, 3, 4]],
    [[1, 3], [0.01, 0.02, 0.03, 0.04]],
    [[3, 3], [100, 200, 300, 400]]
]
num_species = 3
num_pairs = 3
max_sim_epochs = 1000
sim_stop_time = 10.0
time_step = 0.1
individual_fix_size = True


population = population_initialization(
    population_size=population_size,
    individual_shape=individual_shape,
    species_parameters=species_parameters,
    complex_parameters=complex_parameters,
    num_species=num_species,
    num_pairs=num_pairs,
    max_sim_epochs=max_sim_epochs,
    sim_stop_time=sim_stop_time,
    time_step=time_step,
    individual_fix_size=individual_fix_size
)



0
1
2
0
1
2


In [65]:
for i in population:
    print(i.shape)

(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)
(13, 10, 10)


In [66]:
for i in population:
    print(i[-1, -1, :])

[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[3.e+00 3.e+00 1.e+03 1.e+01 1.e-01 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]


In [69]:
population = population_initialization(
    population_size=3,
    individual_shape=(10, 5, 5),
    species_parameters=species_parameters,
    complex_parameters=complex_parameters,
    num_species=num_species,
    num_pairs=num_pairs,
    max_sim_epochs=max_sim_epochs,
    sim_stop_time=sim_stop_time,
    time_step=time_step,
    individual_fix_size=False
)

In [70]:
for i in population:
    print(i)

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]]

 [[4.74926766e-01 4.05502571e-01 5.78077869e-01 9.91683536e-01
   5.68172047e-01]
  [1.70162770e-01 7.05467450e-02 3.92738199e-01 3.85519731e-01
   3.31228068e-01]
  [8.14439875e-01 5.17639789e-01 2.87238520e-02 9.47820800e-01
   6.49140902e-01]
  [6.94788625e-01 6.47510694e-02 7.58729550e-01 1.40491898e-02
   2.76551987e-01]
  [8.53442307e-01 8.41185563e-01 5.95971268e-01 4.88366449e-01
   8.73509149e-02]]

 [[5.19395270e-01 2.63139095e-01 5.80172849e-01 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.0000000

In [71]:
x = [1,2,3,4]
b = [12, 23, 43, 76]
v = x + b
v

[1, 2, 3, 4, 12, 23, 43, 76]

In [7]:
def apply_mutation(individual, sim_mutation_rate, compartment_mutation_rate, parameter_mutation_rate,
                   insertion_mutation_rate, deletion_mutation_rate, sim_means, sim_std_devs, sim_min_vals, sim_max_vals,
                   compartment_mean, compartment_std, compartment_min_val, compartment_max_val, sim_distribution,
                   compartment_distribution, species_param_means, species_param_stds, species_param_min_vals,
                   species_param_max_vals, complex_param_means, complex_param_stds, complex_param_min_vals,
                   complex_param_max_vals, param_distribution, sim_mutation, compartment_mutation, param_mutation,
                   species_insertion_mutation, species_deletion_mutation):

    if sim_mutation:
        individual = apply_simulation_parameters_mutation(
            individual=individual,
            mutation_rate=sim_mutation_rate,
            means=sim_means,
            std_devs=sim_std_devs,
            min_vals=sim_min_vals,
            max_vals=sim_max_vals,
            distribution=sim_distribution
        )
    if compartment_mutation:
        individual = apply_compartment_mutation(
            individual=individual,
            mutation_rate=compartment_mutation_rate,
            mean=compartment_mean,
            std_dev=compartment_std,
            min_val=compartment_min_val,
            max_val=compartment_max_val,
            distribution=compartment_distribution
        )
    if param_mutation:
        individual = apply_parameters_mutation(
            individual=individual,
            mutation_rate=parameter_mutation_rate,
            species_means=species_param_means,
            species_std_devs=species_param_stds,
            species_min_vals=species_param_min_vals,
            species_max_vals=species_param_max_vals,
            complex_means=complex_param_means,
            complex_std_devs=complex_param_stds,
            complex_min_vals=complex_param_min_vals,
            complex_max_vals=complex_param_max_vals,
            distribution=param_distribution
        )
    if species_insertion_mutation:
        individual = apply_species_insertion_mutation(
            individual=individual,
            mutation_rate=insertion_mutation_rate
        )
    if species_deletion_mutation and individual.shape[0] > 3:
        individual = apply_species_deletion_mutation(
            individual=individual,
            mutation_rate=deletion_mutation_rate
        )

    return individual


def apply_simulation_parameters_mutation(individual, mutation_rate, means, std_devs, min_vals, max_vals, distribution):


    mut_mask = np.random.rand(2) < mutation_rate

    for i in range(2):

        if distribution == "uniform":
            individual[-1, -1, i + 3] += (np.random.uniform(low=min_vals[i], high=max_vals[i]) - individual[
                -1, -1, i + 3]) * mut_mask[i]
        elif distribution == "normal":
            individual[-1, -1, i + 3] += np.random.normal(loc=means[i], scale=std_devs[i]) * mut_mask[i]

        individual[-1, -1, i + 3] = max(min_vals[i], min(max_vals[i], individual[-1, -1, i + 3]))

    return individual


def apply_compartment_mutation(individual, mutation_rate, mean, std_dev, min_val, max_val, distribution):

    num_species = int(individual[-1, -1, 0])
    z, y, x = individual.shape

    for i in range(1, num_species * 2, 2):
        mut_mask = np.random.rand(y, x) < mutation_rate

        if distribution == "normal":
            noise = np.random.normal(loc=mean, scale=std_dev, size=(y, x))
        elif distribution == "uniform":
            noise = np.random.uniform(low=min_val, high=max_val, size=(y, x))

        individual[i, :, :] += np.where(mut_mask, noise, 0)
        individual[i, :, :] = np.clip(individual[i, :, :], min_val, max_val)

    return individual



def apply_parameters_mutation(individual, mutation_rate, species_means, species_std_devs,
                             species_min_vals, species_max_vals, complex_means, complex_std_devs,
                             complex_min_vals, complex_max_vals, distribution):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = num_species * 2
    pair_stop = pair_start + (num_pairs * 2)

    count = 0
    for i in range(0, num_species * 2, 2):
        mut_mask = np.random.rand(3) < mutation_rate
        if distribution == "normal":
            for j in range(3):
                individual[-1, i, j] += np.random.normal(loc=species_means[count], scale=species_std_devs[count]) * \
                                        mut_mask[j]
        elif distribution == "uniform":
            for j in range(3):
                individual[-1, i, j] += (np.random.uniform(low=species_min_vals[count], high=species_max_vals[count]) -
                                         individual[-1, i, j]) * mut_mask[j]
        count += 1

    for i in range(pair_start + 1, pair_stop, 2):
        mut_mask = np.random.rand(4) < mutation_rate
        if distribution == "normal":
            for j in range(4):
                individual[i, 1, j] += np.random.normal(loc=complex_means[count], scale=complex_std_devs[count]) * \
                                       mut_mask[j]
        elif distribution == "uniform":
            for j in range(4):
                individual[i, 1, j] += (np.random.uniform(low=complex_min_vals[count], high=complex_max_vals[count]) -
                                        individual[i, 1, j]) * mut_mask[j]
        count += 1

    return individual




def apply_species_insertion_mutation(individual, mutation_rate):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    z, y, x = individual.shape

    if np.random.rand() < mutation_rate:
        pairs = pair_finding(
            num_species=num_species
        )
        init_matrix = species_initialization(
            compartment_size=(y, x),
            pairs=pairs
        )
        individual = species_combine(
            individual=individual,
            init_matrix=init_matrix,
            num_species=num_species,
            num_pairs=num_pairs
        )

    return individual



def apply_species_deletion_mutation(individual, mutation_rate):
  
    num_species = int(individual[-1, -1, 0])

    if np.random.rand() < mutation_rate and num_species > 1:
        deleted_species = int(np.random.choice(np.arange(2, num_species+1)))

        individual = species_deletion(
            individual=individual,
            deleted_species=deleted_species
        )

    return individual




def pair_finding(num_species):

    last = num_species + 1
    species = [i for i in range(1, num_species + 2, 1)]
    pairs = list(itertools.combinations(species, 2))

    related_pairs = [pair for pair in pairs if last in pair]
    pair_indices = [((pair[0] - 1) * 2, (pair[1]-1)*2) for pair in related_pairs]

    return pair_indices




def species_combine(individual, init_matrix, num_species, num_pairs):
  
    z, y, x = individual.shape
    z1 = z + init_matrix.shape[0]

    updated_individual = np.zeros((z1, y, x))
    updated_individual[:num_species * 2, :, :] = individual[:num_species * 2, :, :]
    updated_individual[num_species * 2:num_species * 2 + init_matrix.shape[0], :, :] = init_matrix
    updated_individual[num_species * 2 + init_matrix.shape[0]:, :, :] = individual[num_species * 2:, :, :]
    updated_individual[-1, -1, 0] = int(num_species + 1)
    updated_individual[-1, -1, 1] = int(num_pairs + ((init_matrix.shape[0] - 2) / 2))
    updated_individual[-1, num_species * 2, :3] = np.random.rand(3)

    return updated_individual



def species_deletion(individual, deleted_species):
  
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int((num_species * 2) + 1)
    pair_stop = int(pair_start + (num_pairs * 2))

    delete_indices = [(deleted_species-1)*2, ((deleted_species-1)*2)+1]

    for i in range(pair_start, pair_stop, 2):
        if int((individual[i, 0, 0] / 2) + 1) == deleted_species or int((individual[i, 0, 1] / 2) + 1) == deleted_species:
            delete_indices.extend([i - 1, i])

    updated_individual = np.delete(individual, delete_indices, axis=0)

    updated_individual[-1, -1, 0] = num_species - 1
    updated_individual[-1, -1, 1] = num_pairs - len(delete_indices) // 2 + 1

    return updated_individual


def species_initialization(compartment_size, pairs):
  
    num_species = len(pairs) + 1
    num_matrices = num_species * 2
    init_matrix = np.zeros((num_matrices, compartment_size[0], compartment_size[1]))

    for i in range(len(pairs)):
        m = np.zeros((2, compartment_size[0], compartment_size[1]))
        m[-1, 0, 0] = int(pairs[i][0])
        m[-1, 0, 1] = int(pairs[i][1])
        m[-1, 1, :4] = np.random.rand(4)
        init_matrix[i*2+2:i*2+4, :, :] = m

    return init_matrix

In [10]:
ind = np.zeros((7, 10, 10))
ind[-1, 0, :3] = [.5, .4, 2]
ind[-1, 2, :3] = [.6, .3, 1.5]
ind[-1, -1, :5] = [2, 1, 1000, 20, 0.001]
ind[-2, 0, :2] = [0, 2]
ind[-2, 1, :4] = [.5, .1, .3, 10]
print(ind)

ind = apply_mutation(
    individual=ind,
    sim_mutation_rate=0.5,
    compartment_mutation_rate=0.2,
    parameter_mutation_rate=0.3,
    insertion_mutation_rate=0.6,
    deletion_mutation_rate=0.4,
    sim_means=(0, 0),
    sim_std_devs=(200, 0.1),
    sim_min_vals=(50, 0.0001),
    sim_max_vals=[1000, 0.5],
    compartment_mean=10,
    compartment_std=5,
    compartment_min_val=0,
    compartment_max_val=100,
    sim_distribution="normal",
    compartment_distribution="normal",
    species_param_means=(0.5, 0.1, 5),
    species_param_stds=[.3, .2, 3],
    species_param_min_vals=[0, 0, 0],
    species_param_max_vals=[10, 1, 1],
    complex_param_means=[.5, .1, .2, 100],
    complex_param_stds=[.4, .2, .3, 50],
    complex_param_min_vals=[0, 0, 0, 0],
    complex_param_max_vals=[1, 1, 1, 1000],
    param_distribution="normal",
    sim_mutation=True, 
    compartment_mutation=True,
    param_mutation=True,
    species_insertion_mutation=True,
    species_deletion_mutation=True
)
print(ind.shape)
print(ind)


[[[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]]

 [[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+0

In [11]:
ind = np.zeros((7, 10, 10))
ind[-1, 0, :3] = [.5, .4, 2]
ind[-1, 2, :3] = [.6, .3, 1.5]
ind[-1, -1, :5] = [2, 1, 1000, 20, 0.001]
ind[-2, 0, :2] = [0, 2]
ind[-2, 1, :4] = [.5, .1, .3, 10]
print(ind)

ind = apply_mutation(
    individual=ind,
    sim_mutation_rate=0.5,
    compartment_mutation_rate=0.2,
    parameter_mutation_rate=0.3,
    insertion_mutation_rate=0.6,
    deletion_mutation_rate=0.4,
    sim_means=(200, 0.001),
    sim_std_devs=(50, 0.01),
    sim_min_vals=(50, 0.0001),
    sim_max_vals=[1000, 0.5],
    compartment_mean=10,
    compartment_std=5,
    compartment_min_val=0,
    compartment_max_val=100,
    sim_distribution="uniform",
    compartment_distribution="uniform",
    species_param_means=(0.5, 0.1, 5),
    species_param_stds=[.3, .2, 3],
    species_param_min_vals=[0, 0, 0],
    species_param_max_vals=[10, 1, 1],
    complex_param_means=[.5, .1, .2, 100],
    complex_param_stds=[.4, .2, .3, 50],
    complex_param_min_vals=[0, 0, 0, 0],
    complex_param_max_vals=[1, 1, 1, 1000],
    param_distribution="uniform",
    sim_mutation=True,
    compartment_mutation=True,
    param_mutation=True,
    species_insertion_mutation=True,
    species_deletion_mutation=True
)
print(ind.shape)
print(ind)

[[[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]]

 [[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+0

In [24]:
def apply_crossover(elite_individuals, individual, crossover_alpha, sim_crossover, compartment_crossover, param_crossover):
 
    elite_individual = random.choice(elite_individuals)


    if sim_crossover:
        individual = apply_simulation_variable_crossover(
            elite_individual=elite_individual,
            individual=individual,
            alpha=crossover_alpha
        )
    if compartment_crossover:
        individual = apply_compartment_crossover(
            elite_individual=elite_individual,
            individual=individual,
            alpha=crossover_alpha
        )

    if param_crossover:
        individual = apply_parameter_crossover(
            elite_individual=elite_individual,
            individual=individual,
            alpha=crossover_alpha
        )

    return individual



def apply_simulation_variable_crossover(elite_individual, individual, alpha):

    individual[-1, -1, 3:5] = (alpha * individual[-1, -1, 3:5]) + ((1 - alpha) * elite_individual[-1, -1, 3:5])

    return individual


def apply_compartment_crossover(elite_individual, individual, alpha):
  
    num_species = int(individual[-1, -1, 0])

    for i in range(1, num_species*2+1, 2):
        individual[i, :, :] = (alpha * individual[i, :, :]) + ((1 - alpha) * elite_individual[i, :, :])

    return individual


def apply_parameter_crossover(elite_individual, individual, alpha):
   
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int(num_species * 2)
    pair_stop = int(pair_start + (num_pairs * 2))

    for i in range(0, num_species*2, 2):
        individual[-1, i, :3] = (alpha * individual[-1, i, :3]) + ((1 - alpha) * elite_individual[-1, i, :3])

    for i in range(pair_start+1, pair_stop+1, 2):
        individual[i, 1, :4] = (alpha * individual[i, 1, :4]) + ((1 - alpha) * elite_individual[i, 1, :4])

    return individual




def filter_elite_individuals(low_cost_individuals, elite_individuals, high_cost_individual):
  
    filtered_elite_individuals = [ind for ind in elite_individuals if ind.shape[0] == high_cost_individual.shape[0]]
    if len(filtered_elite_individuals) == 0:
        filtered_elite_individuals = [ind for ind in low_cost_individuals if ind.shape[0] == high_cost_individual.shape[0]]

    return filtered_elite_individuals

In [25]:
ind = np.zeros((7, 10, 10))
ind1 = np.zeros((7, 10, 10))
ind1[1, :, :] = np.full((10, 10), fill_value=12)
ind1[3, :, :] = np.full((10, 10), fill_value=44)
ind1[-1, 0, :3] = [10, 10, 10]
ind1[-1, 2, :3] = [100, 100, 100]
ind1[-1, -1, :5] = [2, 1, 10, 200, 0.1]
ind1[-2, 0, :2] = [0, 2]
ind1[-2, 1, :4] = [55, 11, 11, 1111]

inds = [ind1, ind1, ind1, ind1, ind1]

ind[-1, 0, :3] = [.5, .4, 2]
ind[-1, 2, :3] = [.6, .3, 1.5]
ind[-1, -1, :5] = [2, 1, 1000, 20, 0.001]
ind[-2, 0, :2] = [0, 2]
ind[-2, 1, :4] = [.5, .1, .3, 10]
print(ind)
ind = apply_crossover(
    elite_individuals=inds,
    individual=ind,
    crossover_alpha=0.2,
    sim_crossover=True,
    compartment_crossover=True,
    param_crossover=True
)
ind

[[[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]]

 [[0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00
   0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+0

array([[[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.

In [6]:
target = np.random.rand(10, 10)*10
target

array([[3.42414624, 4.73195021, 9.64818634, 4.84194219, 1.7457221 ,
        2.31817419, 8.13905694, 4.93943943, 3.35300825, 1.00892059],
       [7.42584927, 1.0069079 , 7.13365344, 5.88580752, 2.7631979 ,
        3.89407974, 5.00217508, 1.67112677, 8.73646536, 4.73847736],
       [4.91255411, 6.41431131, 6.38293946, 4.81287738, 5.57815142,
        2.9020899 , 3.72261106, 5.73939503, 2.88716212, 8.53065723],
       [1.48273689, 7.06450688, 3.78837678, 6.11926701, 7.51848558,
        5.97195082, 3.29410349, 5.65110821, 7.98248117, 5.57158552],
       [0.76648464, 2.75585863, 7.98730162, 6.53479499, 4.62222962,
        7.90407598, 0.98757215, 7.87429189, 3.78213319, 5.39308602],
       [0.26847349, 0.42210056, 6.39126252, 5.20974756, 4.64551177,
        7.3007102 , 6.33733801, 0.14517608, 3.91888584, 3.88759554],
       [3.51221718, 2.77749643, 5.79652891, 0.47401271, 2.58297117,
        6.63435135, 8.37231155, 4.03883855, 6.02828163, 8.67350794],
       [1.40326469, 4.73997584, 1.2827152

In [7]:
target.shape

(10, 10)

In [8]:
ind = np.zeros((3, 10, 10))
ind[1, :, :] = np.random.rand(10, 10)
ind[-1, 0, :3] = [.5, .4, 2]
ind[-1, -1, :5] = [1, 0, 1000, 10, 0.01]
print(ind)

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000

In [7]:
f = np.zeros((10, 10))
f[:, 0:3] = 6
f[:, 7:] = 12
f.shape

(10, 10)

In [8]:
h = f.reshape(1, f.shape[0], f.shape[1])
h.shape

(1, 10, 10)

In [9]:
h

array([[[ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.],
        [ 6.,  6.,  6.,  0.,  0.,  0.,  0., 12., 12., 12.]]])

In [12]:
k = np.zeros((1, 1, 1))
k[0, 0, 0] = 1.3
k.shape

(1, 1, 1)

In [13]:
k

array([[[1.3]]])

In [14]:
print(float(k))

1.3


/tmp/ipykernel_5228/2472148024.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(float(k))


In [16]:
n = np.zeros(10)
n[0] = 19
n[1] = 20
n.shape
print(n)

[19. 20.  0.  0.  0.  0.  0.  0.  0.  0.]


In [17]:
j = n[0]
j

19.0

In [18]:
import tensorflow as tf

2024-08-23 17:50:40.397340: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 17:50:41.052610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [20]:
g = np.array([[1,2,3], [8,9,6]])

params = {
    "sp1": tf.Variable(g[1, :]),
    "sp2": tf.Variable(g[0, :])
}
params

{'sp1': <tf.Variable 'Variable:0' shape=(3,) dtype=int64, numpy=array([8, 9, 6])>,
 'sp2': <tf.Variable 'Variable:0' shape=(3,) dtype=int64, numpy=array([1, 2, 3])>}

In [21]:
var = list(params.values())

In [22]:
var

[<tf.Variable 'Variable:0' shape=(3,) dtype=int64, numpy=array([8, 9, 6])>,
 <tf.Variable 'Variable:0' shape=(3,) dtype=int64, numpy=array([1, 2, 3])>]

In [23]:
def parameter_extraction(individual):
    parameters = {}
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int(num_species * 2)
    pair_stop = int(pair_start + (num_pairs * 2))

    species = 1
    for i in range(0, num_species*2, 2):
        parameters[f"species_{species}"] = tf.Variable(individual[-1, i, 0:3])
        species += 1

    pair = 1
    for j in range(pair_start+1, pair_stop+1, 2):
        parameters[f"pair_{pair}"] = tf.Variable(individual[j, 1, :4])
        pair += 1

    return parameters
    
def update_parameters(individual, parameters):
    
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int(num_species * 2)
    pair_stop = int(pair_start + (num_pairs * 2))

    species = 1
    for i in range(0, num_species * 2, 2):
        individual[-1, i, :3] = parameters[f"species_{species}"]
        species += 1

    pair = 1
    for j in range(pair_start + 1, pair_stop + 1, 2):
        individual[j, 1, :4] = parameters[f"pair_{pair}"]
        pair += 1

    return individual

In [24]:
ind = np.zeros((9, 10, 10))
ind[1, :, 0] = 1
ind[3, :, -1] = 1

ind[-1, 0, :3] = [.09, .007, 1.1]
ind[-1, 2, :3] = [0.09, 0.006, 1.2]
ind[-1, -1, :5] = [2, 2, 1000, 5, .01]
ind[-2, 0, 0:2] = [0, 2]
ind[-2, 1, 0:4] = [6, .01, 0.001, 1.3]
ind[-4, 1, 0:4] = [4, 1, 2, 3]

p = parameter_extraction(ind)
p

{'species_1': <tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([0.09 , 0.007, 1.1  ])>,
 'species_2': <tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([0.09 , 0.006, 1.2  ])>,
 'pair_1': <tf.Variable 'Variable:0' shape=(4,) dtype=float64, numpy=array([4., 1., 2., 3.])>,
 'pair_2': <tf.Variable 'Variable:0' shape=(4,) dtype=float64, numpy=array([6.0e+00, 1.0e-02, 1.0e-03, 1.3e+00])>}

In [29]:
inds = np.zeros((9, 10, 10))
inds[-1, -1, :5] = [2, 2, 1000, 5, .01]
inds = update_parameters(inds, p)

In [30]:
inds

array([[[0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
         0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 0

In [32]:
import tensorflow as tf

In [33]:
tf.executing_eagerly()

True

In [34]:
def compute_cost_(y_hat, target):
        
    y, x = y_hat.shape
    cost = np.mean((target - y_hat) ** 2) / (y * x)

    return cost

In [36]:
y = np.full((10, 10), fill_value=10)
x = np.full((10, 10), fill_value=13)
c = compute_cost_(y, x)
c

0.09